## Тема 10. Тестирование в Python

### Задание 1 
Придумать классы эквивалентности для функции, проверяющей, является ли строка палиндромом. Написать эти тесты и реализовать саму функцию. У вас должно получиться как минимум 4 теста. Попробуйте придерживаться принципа TDD: в начале реализуйте тесты, а затем — искомую функцию.


In [1]:
pip install ipytest

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pytest

Note: you may need to restart the kernel to use updated packages.


In [3]:
import ipytest

ipytest.autoconfig(addopts=["-v"])

In [4]:
%%ipytest

def test_1_add():
    """  Тест провалится"""
    assert add(1, 2) == 4
    
def test_2_add():
    """  Тест пройдет"""
    assert add(1, 2) == 3
    
# Просто проверка, как работает pytest(ipytest) для Jupyter Notebook
def add(a, b):
    return a + b




======================================= test session starts ========================================
platform linux -- Python 3.9.13, pytest-7.1.2, pluggy-1.0.0 -- /home/selaptop/anaconda3/bin/python
cachedir: .pytest_cache
rootdir: /home/selaptop/Documents/magistr/2M Python/tasks
plugins: anyio-3.5.0, mock-3.10.0
collecting ... collected 2 items

t_0e72e557c11c43f09b01f82945ff267f.py::test_1_add FAILED                                     [ 50%]
t_0e72e557c11c43f09b01f82945ff267f.py::test_2_add PASSED                                     [100%]

============================================= FAILURES =============================================
____________________________________________ test_1_add ____________________________________________

    def test_1_add():
        """  Тест провалится"""
>       assert add(1, 2) == 4
E       assert 3 == 4
E        +  where 3 = add(1, 2)

/tmp/ipykernel_47324/250491248.py:3: AssertionError
===================================== short test summar

In [5]:
%%ipytest

import re

def test_1_is_palindrome():
    str = 'А роза упала на лапу Азора'
    actual = is_palindrome(str)
    expected = True
    assert actual == expected


def test_2_is_palindrome():
    str = 'Идем, молод, долом меди.'
    actual = is_palindrome(str)
    expected = True
    assert actual == expected
    
def test_3_is_palindrome():
    str = 'Чин зван мечем навзничь'
    actual = is_palindrome(str)
    expected = True
    assert actual == expected
    
def test_4_is_palindrome():
    str = f"Но не речь,\nа черен он."
    actual = is_palindrome(str)
    expected = True
    assert actual == expected



def is_palindrome(str):
    """
    Определяет является ли строка палиндромом
    Возвращает булевое значение
    Реализация для русского языка
    """
    # 1 привести слова к нижнему регистру и удалить пробелы
    str = str.lower().replace(' ', '')
    # 2 удалить знаки препинания
    str = re.sub(r'[^\w\s]', '', str)
    # 3 удалить ь ъ 
    str = re.sub(r'[ь,ъ]', '', str)
    # 4 обработать переносы строк
    str = str.replace("\r","").replace("\n","")
    # магия из python str[::-1] это slice для string/list/tuple. <object_name>[<start_index>, <stop_index>, <step>] 
    return str == str[::-1]

======================================= test session starts ========================================
platform linux -- Python 3.9.13, pytest-7.1.2, pluggy-1.0.0 -- /home/selaptop/anaconda3/bin/python
cachedir: .pytest_cache
rootdir: /home/selaptop/Documents/magistr/2M Python/tasks
plugins: anyio-3.5.0, mock-3.10.0
collecting ... collected 4 items

t_0e72e557c11c43f09b01f82945ff267f.py::test_1_is_palindrome PASSED                           [ 25%]
t_0e72e557c11c43f09b01f82945ff267f.py::test_2_is_palindrome PASSED                           [ 50%]
t_0e72e557c11c43f09b01f82945ff267f.py::test_3_is_palindrome PASSED                           [ 75%]
t_0e72e557c11c43f09b01f82945ff267f.py::test_4_is_palindrome PASSED                           [100%]

======================================== 4 passed in 0.02s =========================================


### Задание 2
Для классов Designer и Developer из домашнего задания к 7 вебинару реализуйте фикстуры designer и developer соответственно, такие, чтобы они возвращали функцию, принимающую на вход только seniority, и напишите с их использованием как минимум один тест, проверяющий грейд в зависимости от переданного seniority.  
Для name и awards мы делаем заглушки dummy. Инициализируем их константами.  
Пример фикстуры для класса с сигнатурой Customer(name, money):  
``@pytest.fixture
def customer():
  def init_customer(money):
    return Customer(name='Bob',money=money)
  return init_customer
``

Из условий не совсем понятно цель задания. Ведь seniority влияет только на экзепляры класса Developer, а не Designer. Возможно это тест на внимательность или неверно понял условие. Поэтому считаю, что seniority в Designer никак не будет изменять его грейд.


In [6]:
class Employee:
    def __init__(self, name, seniority):
        self.name = name
        self.seniority = seniority
        
        self.grade = 1
    
    def grade_up(self):
        """Повышает уровень сотрудника"""
        self.grade += 1
    
    def publish_grade(self):
        """Публикация результатов аккредитации сотрудников"""
        print(self.name, self.grade)
    
    def check_if_it_is_time_for_upgrade(self):
        pass

In [7]:
class Designer(Employee):
    
    def __init__(self, name, seniority, awards):
        super().__init__(name, seniority)
        self.awards = awards
        
    def check_if_it_is_time_for_upgrade(self):
        # для каждой аккредитации увеличиваем счетчик на 2
        # пока считаем, что все дизайнеры получили межд. награду
        self.awards += 2
        
        # условие повышения дизайнера
        if self.awards / 7 > self.grade:
            self.grade_up()
        
        # публикация результатов
        return self.publish_grade()

In [8]:
class Developer(Employee):
    def __init__(self, name, seniority):
        super().__init__(name, seniority)
    
    def check_if_it_is_time_for_upgrade(self):
        # для каждой аккредитации увеличиваем счетчик на 1
        # пока считаем, что все разработчики проходят аккредитацию
        self.seniority += 1
        
        # условие повышения сотрудника из презентации
        if self.seniority % 5 == 0:
            self.grade_up()
        
        # публикация результатов
        return self.publish_grade()

In [9]:
import pytest


@pytest.fixture
def designer():
  def init_designer(awards):
    return Designer(name='Alice', seniority=0, awards=awards)
  return init_designer

@pytest.fixture
def developer():
  def init_developer(seniority):
    return Developer(name='Bob', seniority=seniority)
  return init_developer

In [10]:
%%ipytest

@pytest.mark.usefixtures("developer")
def test_developer(developer):
    """метод тестирования грейда разработчика"""
    dev = developer(seniority=2)
    assert dev.grade == 1

======================================= test session starts ========================================
platform linux -- Python 3.9.13, pytest-7.1.2, pluggy-1.0.0 -- /home/selaptop/anaconda3/bin/python
cachedir: .pytest_cache
rootdir: /home/selaptop/Documents/magistr/2M Python/tasks
plugins: anyio-3.5.0, mock-3.10.0
collecting ... collected 1 item

t_0e72e557c11c43f09b01f82945ff267f.py::test_developer PASSED                                 [100%]

======================================== 1 passed in 0.01s =========================================


### Задание 3
Напишите тест, проверяющий работу метода make_format класса Rate из домашнего задания к вебинару 7.  
При этом тесты должны покрывать все возможные варианты инициализации класса разными значениями переданных аргументов (format='full'/'value', diff=True/False).   
Подсказка: количество написанного кода можно сократить с помощью параметризации тестов, если найти закономерность.  
**!!! Важно! Тест не должен обращаться к внешним ресурсам. Вам необходимо их “замокать” (запрос к ресурсу с курсами валют)**  

Все валюты из источника тесты проверять не нужно, достаточно выбрать одну валюту.  


In [13]:
import requests
import json
from unittest.mock import MagicMock, patch

In [14]:
class Rate:
    def __init__(self, format_='value', diff_=False):
        self.format = format_
        self.diff = diff_

    def exchange_rates(self):
        """
        Возвращает ответ сервиса с инфой о валютах в виде:
        {
        "AUD": {
            "ID": "R01010",
            "NumCode": "036",
            "CharCode": "AUD",
            "Nominal": 1,
            "Name": "Австралийский доллар",
            "Value": 38.814,
            "Previous": 38.3392
        },
        "AZN": {
            "ID": "R01020A",
            "NumCode": "944",
            "CharCode": "AZN",
            "Nominal": 1,
            "Name": "Азербайджанский манат",
            "Value": 35.9782,
            "Previous": 35.9975
        },
        }
        """
        self.r = requests.get('https://www.cbr-xml-daily.ru/daily_json.js')
        return self.r.json()['Valute']

    def make_format(self, currency):
        """
        Return info about currency in two variants:
        1. full info about currency if self.format = 'full'
        Rate('full').make_format('AZN')
        {
            "ID": "R01020A",
            "NumCode": "944",
            "CharCode": "AZN",
            "Nominal": 1,
            "Name": "Азербайджанский манат",
            "Value": 35.9782,
            "Previous": 35.9975
        }

        2. Short info about currency
        Rate('value').make_format('AZN')
        35.9782
        """
        response = self.exchange_rates()

        if currency in response:
            if self.format == 'full':
                return response[currency]

            if self.format == 'value':
                if self.diff:
                    return response[currency]['Value'] - response[currency]['Previous']
                return response[currency]['Value']

        return 'Error'

    def eur(self):
        """Return exchange rate euro in self.format"""
        return self.make_format('EUR')

    def usd(self):
        """Return exchange rate euro in self.format"""
        return self.make_format('USD')

    def brl(self):
        """Return exchange rate euro in self.format"""
        return self.make_format('BRL')

    def max_rate(self):
        """Return name of valute of max Value/Nominal"""
        response = self.exchange_rates()
        name = ''
        value = 0
        for el in response:
            ye = response[el]['Value'] / response[el]['Nominal']
            if ye > value:
                value = ye
                name = el
        return name

In [22]:
mocked_json = {
    "Date": "2022-11-15T11:30:00+03:00",
    "PreviousDate": "2022-11-12T11:30:00+03:00",
    "PreviousURL": "\/\/www.cbr-xml-daily.ru\/archive\/2022\/11\/12\/daily_json.js",
    "Timestamp": "2022-11-15T15:00:00+03:00",
    "Valute": {
        "AUD": {
            "ID": "R01010",
            "NumCode": "036",
            "CharCode": "AUD",
            "Nominal": 1,
            "Name": "Австралийский доллар",
            "Value": 40.3581,
            "Previous": 39.8703
        },
        "AZN": {
            "ID": "R01020A",
            "NumCode": "944",
            "CharCode": "AZN",
            "Nominal": 1,
            "Name": "Азербайджанский манат",
            "Value": 35.5284,
            "Previous": 35.4223
        },
        "GBP": {
            "ID": "R01035",
            "NumCode": "826",
            "CharCode": "GBP",
            "Nominal": 1,
            "Name": "Фунт стерлингов Соединенного королевства",
            "Value": 71.1128,
            "Previous": 70.3225
        },
        "AMD": {
            "ID": "R01060",
            "NumCode": "051",
            "CharCode": "AMD",
            "Nominal": 100,
            "Name": "Армянских драмов",
            "Value": 15.2548,
            "Previous": 15.2019
        },
        "BYN": {
            "ID": "R01090B",
            "NumCode": "933",
            "CharCode": "BYN",
            "Nominal": 1,
            "Name": "Белорусский рубль",
            "Value": 24.9157,
            "Previous": 24.8137
        },
        "BGN": {
            "ID": "R01100",
            "NumCode": "975",
            "CharCode": "BGN",
            "Nominal": 1,
            "Name": "Болгарский лев",
            "Value": 31.8321,
            "Previous": 30.6468
        },
        "BRL": {
            "ID": "R01115",
            "NumCode": "986",
            "CharCode": "BRL",
            "Nominal": 1,
            "Name": "Бразильский реал",
            "Value": 11.3856,
            "Previous": 11.3488
        },
        "HUF": {
            "ID": "R01135",
            "NumCode": "348",
            "CharCode": "HUF",
            "Nominal": 100,
            "Name": "Венгерских форинтов",
            "Value": 15.3361,
            "Previous": 15.3207
        },
        "HKD": {
            "ID": "R01200",
            "NumCode": "344",
            "CharCode": "HKD",
            "Nominal": 10,
            "Name": "Гонконгских долларов",
            "Value": 77.2059,
            "Previous": 76.9656
        },
        "DKK": {
            "ID": "R01215",
            "NumCode": "208",
            "CharCode": "DKK",
            "Nominal": 10,
            "Name": "Датских крон",
            "Value": 83.6992,
            "Previous": 80.586
        },
        "USD": {
            "ID": "R01235",
            "NumCode": "840",
            "CharCode": "USD",
            "Nominal": 1,
            "Name": "Доллар США",
            "Value": 60.3982,
            "Previous": 60.2179
        },
        "EUR": {
            "ID": "R01239",
            "NumCode": "978",
            "CharCode": "EUR",
            "Nominal": 1,
            "Name": "Евро",
            "Value": 62.1554,
            "Previous": 61.5416
        },
        "INR": {
            "ID": "R01270",
            "NumCode": "356",
            "CharCode": "INR",
            "Nominal": 100,
            "Name": "Индийских рупий",
            "Value": 73.4671,
            "Previous": 72.966
        },
        "KZT": {
            "ID": "R01335",
            "NumCode": "398",
            "CharCode": "KZT",
            "Nominal": 100,
            "Name": "Казахстанских тенге",
            "Value": 13.0729,
            "Previous": 12.8281
        },
        "CAD": {
            "ID": "R01350",
            "NumCode": "124",
            "CharCode": "CAD",
            "Nominal": 1,
            "Name": "Канадский доллар",
            "Value": 45.1474,
            "Previous": 45.0126
        },
        "KGS": {
            "ID": "R01370",
            "NumCode": "417",
            "CharCode": "KGS",
            "Nominal": 100,
            "Name": "Киргизских сомов",
            "Value": 71.5774,
            "Previous": 71.4323
        },
        "CNY": {
            "ID": "R01375",
            "NumCode": "156",
            "CharCode": "CNY",
            "Nominal": 10,
            "Name": "Китайских юаней",
            "Value": 85.4248,
            "Previous": 84.4637
        },
        "MDL": {
            "ID": "R01500",
            "NumCode": "498",
            "CharCode": "MDL",
            "Nominal": 10,
            "Name": "Молдавских леев",
            "Value": 31.3931,
            "Previous": 31.2715
        },
        "NOK": {
            "ID": "R01535",
            "NumCode": "578",
            "CharCode": "NOK",
            "Nominal": 10,
            "Name": "Норвежских крон",
            "Value": 60.2266,
            "Previous": 60.3446
        },
        "PLN": {
            "ID": "R01565",
            "NumCode": "985",
            "CharCode": "PLN",
            "Nominal": 1,
            "Name": "Польский злотый",
            "Value": 13.2371,
            "Previous": 13.2408
        },
        "RON": {
            "ID": "R01585F",
            "NumCode": "946",
            "CharCode": "RON",
            "Nominal": 1,
            "Name": "Румынский лей",
            "Value": 12.6919,
            "Previous": 12.6391
        },
        "XDR": {
            "ID": "R01589",
            "NumCode": "960",
            "CharCode": "XDR",
            "Nominal": 1,
            "Name": "СДР (специальные права заимствования)",
            "Value": 79.0673,
            "Previous": 77.3806
        },
        "SGD": {
            "ID": "R01625",
            "NumCode": "702",
            "CharCode": "SGD",
            "Nominal": 1,
            "Name": "Сингапурский доллар",
            "Value": 44.0284,
            "Previous": 43.5132
        },
        "TJS": {
            "ID": "R01670",
            "NumCode": "972",
            "CharCode": "TJS",
            "Nominal": 10,
            "Name": "Таджикских сомони",
            "Value": 59.1461,
            "Previous": 58.9672
        },
        "TRY": {
            "ID": "R01700J",
            "NumCode": "949",
            "CharCode": "TRY",
            "Nominal": 10,
            "Name": "Турецких лир",
            "Value": 32.598,
            "Previous": 32.3822
        },
        "TMT": {
            "ID": "R01710A",
            "NumCode": "934",
            "CharCode": "TMT",
            "Nominal": 1,
            "Name": "Новый туркменский манат",
            "Value": 17.2566,
            "Previous": 17.2051
        },
        "UZS": {
            "ID": "R01717",
            "NumCode": "860",
            "CharCode": "UZS",
            "Nominal": 10000,
            "Name": "Узбекских сумов",
            "Value": 53.8938,
            "Previous": 53.7907
        },
        "UAH": {
            "ID": "R01720",
            "NumCode": "980",
            "CharCode": "UAH",
            "Nominal": 10,
            "Name": "Украинских гривен",
            "Value": 16.354,
            "Previous": 16.304
        },
        "CZK": {
            "ID": "R01760",
            "NumCode": "203",
            "CharCode": "CZK",
            "Nominal": 10,
            "Name": "Чешских крон",
            "Value": 25.6414,
            "Previous": 24.6038
        },
        "SEK": {
            "ID": "R01770",
            "NumCode": "752",
            "CharCode": "SEK",
            "Nominal": 10,
            "Name": "Шведских крон",
            "Value": 58.1322,
            "Previous": 57.1235
        },
        "CHF": {
            "ID": "R01775",
            "NumCode": "756",
            "CharCode": "CHF",
            "Nominal": 1,
            "Name": "Швейцарский франк",
            "Value": 63.8661,
            "Previous": 62.7728
        },
        "ZAR": {
            "ID": "R01810",
            "NumCode": "710",
            "CharCode": "ZAR",
            "Nominal": 10,
            "Name": "Южноафриканских рэндов",
            "Value": 34.9824,
            "Previous": 34.8487
        },
        "KRW": {
            "ID": "R01815",
            "NumCode": "410",
            "CharCode": "KRW",
            "Nominal": 1000,
            "Name": "Вон Республики Корея",
            "Value": 45.5526,
            "Previous": 45.675
        },
        "JPY": {
            "ID": "R01820",
            "NumCode": "392",
            "CharCode": "JPY",
            "Nominal": 100,
            "Name": "Японских иен",
            "Value": 43.2652,
            "Previous": 42.273
        }
    }
}

In [25]:
%%ipytest

@patch('requests.get')
@pytest.mark.parametrize("format_val, diff_val, expected",
                         [("value", False, 35.5284), ("value", True, 0.10609999999999786), ("full", False, {
                             "ID": "R01020A",
                             "NumCode": "944",
                             "CharCode": "AZN",
                             "Nominal": 1,
                             "Name": "Азербайджанский манат",
                             "Value": 35.5284,
                             "Previous": 35.4223
                         }), ("full", True, {
                             "ID": "R01020A",
                             "NumCode": "944",
                             "CharCode": "AZN",
                             "Nominal": 1,
                             "Name": "Азербайджанский манат",
                             "Value": 35.5284,
                             "Previous": 35.4223
                         })])
def test_make_format(request_mock, format_val, diff_val, expected):
    """Тестирует метод make_format класса Rate, с без отправки запроса на сервер,
    т.е. с замоканным ответом при использовании библиотеки requests """
    # замокаем requests
    response_mock = MagicMock()
    response_mock.status_code = 200
    response_mock.json.return_value = mocked_json

    request_mock.return_value = response_mock

    res = Rate(format_=format_val, diff_=diff_val).make_format("AZN")
    assert res == expected

======================================= test session starts ========================================
platform linux -- Python 3.9.13, pytest-7.1.2, pluggy-1.0.0 -- /home/selaptop/anaconda3/bin/python
cachedir: .pytest_cache
rootdir: /home/selaptop/Documents/magistr/2M Python/tasks
plugins: anyio-3.5.0, mock-3.10.0
collecting ... collected 4 items

t_0e72e557c11c43f09b01f82945ff267f.py::test_make_format[value-False-35.5284] PASSED          [ 25%]
t_0e72e557c11c43f09b01f82945ff267f.py::test_make_format[value-True-0.10609999999999786] PASSED [ 50%]
t_0e72e557c11c43f09b01f82945ff267f.py::test_make_format[full-False-expected2] PASSED         [ 75%]
t_0e72e557c11c43f09b01f82945ff267f.py::test_make_format[full-True-expected3] PASSED          [100%]

======================================== 4 passed in 0.02s =========================================
